In [1]:
import numpy as np
import re
from nltk.corpus import stopwords
import csv


# The `stop` is defined as earlier in this chapter
# Added it here for convenience, so that this section
# can be run as standalone without executing prior code
# in the directory
stop = stopwords.words('german')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    #tokenized = [w for w in text.split()]
    return tokenized


def stream_docs(path):
    #with open(path, 'rb') as csvfile:
   #     csv_reader = csv.reader(csvfile, delimiter=';', quotechar='\'')
   #     for line in csv_reader:
    #        text, label = line[:-3], int(line[-2])
    #        yield text, label     
    
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            print(line)
            text, label = line[:-3], int(line[-2])
            yield text, label            

In [2]:
next(stream_docs(path='trainingsdata/training.csv'))

'Das war schon vor Jahren mein Lied!!!!';1



("'Das war schon vor Jahren mein Lied!!!!'", 1)

In [3]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [5]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
else:
    clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

doc_stream = stream_docs(path='trainingsdata/training.csv')

In [6]:
#Only needed for progressbar
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=100)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

C:\Users\Matthias\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
0% [#                             ] 100% | ETA: 00:00:01

'Das war schon vor Jahren mein Lied!!!!';1

'Super geil der alte Haudegen wie in alten Zeiten,super';1

'Udo,du bist der schärfste auf diesem Planeten ! Mach weiter so,immer weiter';1

'Ich muß mal ganz ehrlich sagen das ich die neue Version des liedes überhaupt net gut finde…*kopfschüttel* ansonsten is udo echt der Hammer.';1

'läuft seit Donnerstag dank Amazon bei mir im Player rauf & runter !Danke Udo geile Scheibe !Doppelzimmer Edition meine Empfehlung an alle die noch überlegen!';1

'Lief bereits auf VIVA-HD...sehr geil, der alte Mann kann´s noch. Nächsten Monat kommt die Blu-ray.';1

'"""is ja echt erstaunlich!!Da wohnt Udo ein paar Wochen nicht im Atlantic und schon hat das Hotel seine ”Sterne” wieder ';-)! Mensch Udo was treibst du denn da so eigendlich???? :-)"""';1

'Das Album ist der absolute Oberhammer! lindenberg beweist wie kein zweiter sein Gespür für tiefgründige Lyrik und Poesie. Auch zeigt er den ganzen Eintagsfliegen im Showgeschäft, was echter Rock bedeutet. Er ist 

ValueError: invalid literal for int() with base 10: '!'

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=500)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
doc_stream = stream_docs(path='german_validation.csv')

X_val, y_val = get_minibatch(doc_stream, size=7)
X_val = vect.transform(X_val)
print('Accuracy: %.3f' % clf.score(X_val, y_val))

In [ ]:
predictions = clf.predict(X_val)
print(predictions)